In [2]:
from src.dnd.units import Unit, GenericSoldier
from src.dnd.game_board import DnDBoard, MovementError
from src.gui.gridDrawer import generate_grid, draw_field, highlight_selected_token
from src.gui.adapters import RenderUnit

from typing import List
import gradio as gr
import itertools
import glob
import re
import numpy as np

/home/genom10/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
board_size = None
board = None
render_units: List[RenderUnit] = []
selectedToken = None

def getUIDsFromBoard(board: DnDBoard):
  return board.get_UIDs()

def update_unit_list() -> None: 
  global board
  global render_units
  UID2unit = dict(zip([unit.get_UID() for unit in board.units], board.units))
  # print(UID2unit)
  # render_units = 

def generate_board(gridScale, gridSize):
  global board_size
  global board
  board_size = gridSize
  board = DnDBoard((board_size, board_size))
  field = generate_grid(board_size, gridScale)
  return field

def add_token(image, x:int, y:int, label, gridScale):
  #TODO: pass name label to Unit
  global board_size
  board.place_unit(GenericSoldier(), (x,y), 0)
  render_units.append(RenderUnit(board.board[(x,y)].get_UID(), (x,y), image))
  if x < board_size:
    x += 1
  else:
    x = 0
    y += 1
  update_unit_list()
  return draw_field(render_units, selectedToken, gridScale, board_size), gr.Number(value=x), gr.Number(value=y), gr.Label(label)

def move_token(index, x:int, y:int, gridScale):
  # x, y = int(x), int(y)
  global board_size
  unit:Unit = board.get_unit_by_UID(render_units[index].getUID())
  # print("unit UID:", unit.get_UID())
  try:
    board.move_unit(unit=unit, new_position=(x,y))
    render_units[index].pos = np.array([x, y], dtype = int)
  except MovementError as e:
    print("Error moving into chosen cell:", e)
  return draw_field(render_units, selectedToken, gridScale, board_size)

def set_selected_token(tokenID, gridScale):
  global board_size
  global selectedToken
  selectedToken = tokenID
  return draw_field(render_units, selectedToken, gridScale, board_size)

def on_grid_scale_shange(gridScale):
  global board_size
  return draw_field(render_units, selectedToken, gridScale, board_size)

def update_UID_list():
  new_choices = list(map(lambda x : x.getUID(), render_units))
  new_choices = list(zip(new_choices, itertools.count()))
  return gr.Dropdown(new_choices, interactive=True, label='tokenID')

def on_image_load(currentLabel):
  #TODO read image label
  if currentLabel == '':
    return 
  else:
    return currentLabel 

Interface

In [4]:
with gr.Blocks() as demo:
  #Grid
  with gr.Row():
    gridSize = gr.Slider(label="gridSize", value=10, minimum=2, maximum=100, step=1)
    makeBoard = gr.Button(value="Make Board")
  gridScale = gr.Slider(label="gridScale", value=30, minimum=10, maximum=150, step=1)
  im_canvas = gr.Image()
  makeBoard.click(generate_board, inputs = [gridScale, gridSize], outputs=[im_canvas])
  gridScale.change(on_grid_scale_shange, inputs = [gridScale], outputs=[im_canvas])

  #Add Token
  with gr.Tabs(visible=True) as tabs:
    with gr.TabItem("Add Token") as tab1:
      with gr.Row():
        with gr.Column(min_width=10):
          x = gr.Number(label="x pos", precision=0)
          y = gr.Number(label="y pos", precision=0)
          label = gr.Text()
        im_in = gr.Image(image_mode='RGBA', type='pil')
        examples_paths = glob.glob('./Tokens/*')
        token_data = [[path, path.split('/')[-1].split('.')[0]] for path in examples_paths]
        examples = gr.Examples(examples=token_data, inputs=[im_in, label], examples_per_page=4)
      btn_add = gr.Button(value="Add token")
    im_in.change(on_image_load, inputs=label, outputs=label)
    btn_add.click(add_token, inputs=[im_in, x, y, label, gridScale], outputs=[im_canvas, x, y, label])

    #Move Token
    with gr.TabItem("Move Token") as tab2:
      with gr.Row():
          tokenID = gr.Dropdown(['t1', 't2'], label='tokenID')
          tab2.select(update_UID_list, outputs=tokenID)
          x_move = gr.Number(label="x pos", precision=0)
          y_move = gr.Number(label="y pos", precision=0)
      btn_move = gr.Button(value="Move token")
    tokenID.change(set_selected_token, inputs=[tokenID, gridScale], outputs=[im_canvas])
    btn_move.click(move_token, inputs=[tokenID, x_move, y_move, gridScale], outputs=[im_canvas])


if __name__ == "__main__":
    demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Error moving into chosen cell: Too far
Error moving into chosen cell: Too far
Error moving into chosen cell: Too far
